<a href="https://colab.research.google.com/github/MyeongheonChoi/BasicSession/blob/main/NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torchviz
!pip install graphviz

  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4150 sha256=c4916a7c751226dd0ad2dc26044c2225663002d89ffa8b9b579e8a3564dcfd7b
  Stored in directory: /root/.cache/pip/wheels/04/38/f5/dc4f85c3909051823df49901e72015d2d750bd26b086480ec2
Successfully built torchviz


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelBinarizer

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score

In [ ]:
def get_f1(test, pred, avg = None):
    confusion = confusion_matrix(test, pred)
    accuracy = accuracy_score(test, pred)
    precision = precision_score(test, pred, average = avg)
    recall = recall_score(test, pred, average = avg)
    f1 = f1_score(test, pred, average = avg)
    print('Confusion Matrix')
    print(confusion)
    print(f'Accuracy : {accuracy}, Precision : {precision}, Recall : {recall}, F1 : {f1}')
    return precision, recall, f1

In [ ]:
a, b, c = get_f1([1,1,1,1,2,2,2,0,0,0,0,0], [0,1,2,0,1,2,0,1,2,0,1,2], avg = 'macro')

Confusion Matrix
[[1 2 2]
 [2 1 1]
 [1 1 1]]
Accuracy : 0.25, Precision : 0.25, Recall : 0.2611111111111111, F1 : 0.2526455026455026


In [ ]:
device=torch.device("cuda:0" if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
w2vaggdata = pd.read_csv("/content/drive/Shareddrives/NLP모델링/코드 연습/문서/w2vaggver.csv")
w2vnormdata = pd.read_csv("/content/drive/Shareddrives/NLP모델링/코드 연습/문서/w2vnormver.csv")
d2vnormdata = pd.read_csv("/content/drive/Shareddrives/NLP모델링/코드 연습/문서/w2vnormver.csv")
w2vaggdata['input'] = [list(map(float, w2vaggdata['input'].iloc[i][1:-1].split(', '))) for i in range(len(w2vaggdata))]
w2vnormdata['input'] = [list(map(float, w2vnormdata['input'].iloc[i][1:-1].split(', '))) for i in range(len(w2vnormdata))]
d2vnormdata['input'] = [list(map(float, d2vnormdata['input'].iloc[i][1:-1].split(', '))) for i in range(len(d2vnormdata))]
w2vaggdata['target'] = [list(map(float, w2vaggdata['target'].iloc[i][1:-1].split(', '))) for i in range(len(w2vaggdata))]
w2vnormdata['target'] = [list(map(float, w2vnormdata['target'].iloc[i][1:-1].split(', '))) for i in range(len(w2vnormdata))]
d2vnormdata['target'] = [list(map(float, d2vnormdata['target'].iloc[i][1:-1].split(', '))) for i in range(len(d2vnormdata))]

In [ ]:
criteria = pd.DataFrame(list(w2vnormdata['target']), columns = ['c1','c2','c3','c4','c5','c6'])
criteria = criteria.astype('str')

In [ ]:
c11 = LabelBinarizer().fit_transform(criteria['c1'])
c12 = LabelBinarizer().fit_transform(criteria['c2'])
c13 = LabelBinarizer().fit_transform(criteria['c3'])
c14 = LabelBinarizer().fit_transform(criteria['c4'])
c15 = LabelBinarizer().fit_transform(criteria['c5'])
c16 = LabelBinarizer().fit_transform(criteria['c6'])

In [ ]:
criteria['c1'] = list(c11)
criteria['c2'] = list(c12)
criteria['c3'] = list(c13)
criteria['c4'] = list(c14)
criteria['c5'] = list(c15)
criteria['c6'] = list(c16)

In [ ]:
class GetDataset(Dataset):
    def __init__(self, data, criteria, index):
        self.x_data = data['input'].values
        self.y_data = criteria[index].values
  
    def __len__(self):
        return len(self.x_data)
  
    def __getitem__(self,idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y

In [ ]:
class nnn(nn.Module):
    def __init__(self):
        super(nnn, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(100,20),
            nn.BatchNorm1d(20),
            nn.ReLU(),
            nn.Linear(20, 3),
            nn.Sigmoid()
        )

    def forward(self, x):
        o1 = self.layer1(x)

        return o1

In [ ]:
aa = nnn().to(device)
summary(aa, (100,), batch_size = 10)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [10, 64]           6,464
       BatchNorm1d-2                   [10, 64]             128
              ReLU-3                   [10, 64]               0
            Linear-4                   [10, 32]           2,080
       BatchNorm1d-5                   [10, 32]              64
              ReLU-6                   [10, 32]               0
            Linear-7                   [10, 16]             528
       BatchNorm1d-8                   [10, 16]              32
              ReLU-9                   [10, 16]               0
           Linear-10                    [10, 8]             136
      BatchNorm1d-11                    [10, 8]              16
             ReLU-12                    [10, 8]               0
           Linear-13                    [10, 3]              27
Total params: 9,475
Trainable params: 9

In [ ]:
x = Variable(torch.randn(1,100)).to(device)
make_dot(aa(x), params=dict(aa.named_parameters()))

In [ ]:
def reset_weights(m):
    for layer in m.children():
        if hasattr(layer, 'reset_parameters'):
            print(f'Reset trainable parameters of layer = {layer}')
            layer.reset_parameters()

In [ ]:
if __name__ == '__main__':

    k_folds = 5
    num_epochs = 50
    loss_function = nn.CrossEntropyLoss()

    results = {}

    # w2v_train1 = GetDataset(w2vaggdata);w2v_test1 = GetDataset(w2vaggdata)
    # w2v_train2 = GetDataset(w2vnormdata);w2v_test2 = GetDataset(w2vnormdata)
    # d2v_train1 = GetDataset(d2vnormdata);d2v_test1 = GetDataset(d2vnormdata)
    # w2v_dataset1 = ConcatDataset([w2v_train1, w2v_test1])
    # w2v_dataset2 = ConcatDataset([w2v_train2, w2v_test2])
    # d2v_dataset1 = ConcatDataset([d2v_train1, d2v_test1])

    train = GetDataset(w2v, criteria, 'c1');test = GetDataset(w2v, criteria, 'c1')
    dataset = ConcatDataset([train, test])     

    kfold = KFold(n_splits = k_folds, shuffle = True)

    for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
        print(f'FOLD {fold+1}')
        print('-' * 30)

        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

        trainloader = DataLoader(dataset, batch_size = 10, sampler = train_subsampler)
        testloader = DataLoader(dataset, batch_size = 10, sampler = test_subsampler)

        model = nnn().to(device)
        model.apply(reset_weights)
    
        optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

        for epoch in range(0, num_epochs):

            print(f'Starting epoch {epoch + 1}')

            current_loss = 0.0
            n = 0
            for i, data in enumerate(trainloader, 0):
                inputs, targets = data
                optimizer.zero_grad()
                outputs = model(inputs.to(device))
                loss = loss_function(outputs, targets.to(device))
                loss.backward()
                optimizer.step()

                current_loss += loss.item()
                n += 1
                
            print('Loss after epoch %5d: %.3f'%(epoch, current_loss / n))
        
        print('Training process has finished. Saving trained model.')

        print('Start Testing')
        save_path = f'/content/drive/Shareddrives/NLP모델링/코드 연습/문서/model-fold-{fold}.pth'
        torch.save(model.state_dict(), save_path)

        correct, total = 0, 0
        with torch.no_grad():
            for i, data in enumerate(testloader, 0):
                inputs, targets = data
                outputs = model(inputs.to(device))
                _, predicted = torch.max(outputs.data, 1)
                _, target = torch.max(targets.data, 1)
                total += targets.size(0)
                correct += (predicted == target.to(device)).sum().item()

            print('Accuracy for fold %d : %d %%' % (fold, 100.0 * correct/total))
            print('-'*30)
            results[fold] = 100.0 * (correct / total)
        
    print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
    print('-'*30)
    sum = 0.0
    for key, value in results.items():
        print(f'Fold {key}: {value} %')
        sum += value
    print(f'Average : {sum/len(results.items())} %')

FOLD 1
------------------------------
Reset trainable parameters of layer = Linear(in_features=100, out_features=64, bias=True)
Reset trainable parameters of layer = BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Linear(in_features=64, out_features=32, bias=True)
Reset trainable parameters of layer = BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=16, bias=True)
Reset trainable parameters of layer = BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=8, bias=True)
Reset trainable parameters of layer = BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Linear(in_features=8, out_features=3, bias=True)
Starting epoch 1
Loss after epoch     0: 1.005
S

In [ ]:
if __name__ == '__main__':

    k_folds = 5
    num_epochs = 50
    loss_function = nn.CrossEntropyLoss()

    results = {}

    # w2v_train1 = GetDataset(w2vaggdata);w2v_test1 = GetDataset(w2vaggdata)
    # w2v_train2 = GetDataset(w2vnormdata);w2v_test2 = GetDataset(w2vnormdata)
    # d2v_train1 = GetDataset(d2vnormdata);d2v_test1 = GetDataset(d2vnormdata)
    # w2v_dataset1 = ConcatDataset([w2v_train1, w2v_test1])
    # w2v_dataset2 = ConcatDataset([w2v_train2, w2v_test2])
    # d2v_dataset1 = ConcatDataset([d2v_train1, d2v_test1])
    for c in [1,2,3,4,5,6]:
        train = GetDataset(w2v, criteria, 'c'+str(c));test = GetDataset(w2v, criteria, 'c'+str(c))
        dataset = ConcatDataset([train, test])     

        kfold = KFold(n_splits = k_folds, shuffle = True)

        for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
            print(f'FOLD {fold+1} for Criteria{c}')
            print('-' * 30)

            train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
            test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

            trainloader = DataLoader(dataset, batch_size = 10, sampler = train_subsampler)
            testloader = DataLoader(dataset, batch_size = 10, sampler = test_subsampler)

            model = nnn().to(device)
            model.apply(reset_weights)
        
            optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

            for epoch in range(0, num_epochs):

                current_loss = 0.0
                n = 0
                for i, data in enumerate(trainloader, 0):
                    inputs, targets = data
                    optimizer.zero_grad()
                    outputs = model(inputs.to(device))
                    loss = loss_function(outputs, targets.to(device))
                    loss.backward()
                    optimizer.step()

                    current_loss += loss.item()
                    n += 1
                    
                print('Loss after epoch %5d: %.3f'%(epoch, current_loss / n))
            
            print('Training process has finished. Saving trained model.')

            print('Start Testing')
            save_path = f'/content/drive/Shareddrives/NLP모델링/코드 연습/문서/model-fold-{fold}-c{c}.pth'
            torch.save(model.state_dict(), save_path)

            correct, total = 0, 0
            with torch.no_grad():
                for i, data in enumerate(testloader, 0):
                    inputs, targets = data
                    outputs = model(inputs.to(device))
                    _, predicted = torch.max(outputs.data, 1)
                    _, target = torch.max(targets.data, 1)
                    total += targets.size(0)
                    correct += (predicted == target.to(device)).sum().item()

                print('Accuracy for fold %d criteria%d: %d %%' % (fold, c, 100.0 * correct/total))
                print('-'*30)
                results[fold] = 100.0 * (correct / total)
            
        print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS for CRITERIA{c}')
        print('-'*30)
        sum = 0.0
        for key, value in results.items():
            print(f'Fold {key} for Criteria{c}: {value} %')
            sum += value
        print(f'Average : {sum/len(results.items())} %')

FOLD 1 for Criteria1
------------------------------
Reset trainable parameters of layer = Linear(in_features=100, out_features=64, bias=True)
Reset trainable parameters of layer = BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Linear(in_features=64, out_features=32, bias=True)
Reset trainable parameters of layer = BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=16, bias=True)
Reset trainable parameters of layer = BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=8, bias=True)
Reset trainable parameters of layer = BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Linear(in_features=8, out_features=3, bias=True)
Loss after epoch     0: 1.457
Loss

In [ ]:
criteria1 = pd.DataFrame(list(w2vaggdata['target']), columns = ['c1','c2','c3','c4','c5','c6'])
criteria1 = criteria1.astype('str')

c21 = LabelBinarizer().fit_transform(criteria1['c1'])
c22 = LabelBinarizer().fit_transform(criteria1['c2'])
c23 = LabelBinarizer().fit_transform(criteria1['c3'])
c24 = LabelBinarizer().fit_transform(criteria1['c4'])
c25 = LabelBinarizer().fit_transform(criteria1['c5'])
c26 = LabelBinarizer().fit_transform(criteria1['c6'])

criteria1['c1'] = list(c21)
criteria1['c2'] = list(c22)
criteria1['c3'] = list(c23)
criteria1['c4'] = list(c24)
criteria1['c5'] = list(c25)
criteria1['c6'] = list(c26)

In [ ]:
if __name__ == '__main__':

    k_folds = 5
    num_epochs = 100
    loss_function = nn.CrossEntropyLoss()


    # w2v_train1 = GetDataset(w2vaggdata);w2v_test1 = GetDataset(w2vaggdata)
    # w2v_train2 = GetDataset(w2vnormdata);w2v_test2 = GetDataset(w2vnormdata)
    # d2v_train1 = GetDataset(d2vnormdata);d2v_test1 = GetDataset(d2vnormdata)
    # w2v_dataset1 = ConcatDataset([w2v_train1, w2v_test1])
    # w2v_dataset2 = ConcatDataset([w2v_train2, w2v_test2])
    # d2v_dataset1 = ConcatDataset([d2v_train1, d2v_test1])
    for c in [1,2,3,4,5,6]:

        results = {}
        pre = []
        rec = []
        f1s = []

        train = GetDataset(w2vaggdata, criteria1, 'c'+str(c));test = GetDataset(w2vaggdata, criteria1, 'c'+str(c))
        dataset = ConcatDataset([train, test])     

        kfold = KFold(n_splits = k_folds, shuffle = True)

        for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
            print(f'FOLD {fold+1} for Criteria{c}')
            print('-' * 30)

            train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
            test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

            trainloader = DataLoader(dataset, batch_size = 10, sampler = train_subsampler)
            testloader = DataLoader(dataset, batch_size = 10, sampler = test_subsampler)

            model = nnn().to(device)
            model.apply(reset_weights)
        
            optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

            for epoch in range(0, num_epochs):

                current_loss = 0.0
                n = 0
                for i, data in enumerate(trainloader, 0):
                    inputs, targets = data
                    optimizer.zero_grad()
                    outputs = model(inputs.to(device))
                    loss = loss_function(outputs, targets.to(device))
                    loss.backward()
                    optimizer.step()

                    current_loss += loss.item()
                    n += 1
                    
                print('Loss after epoch %5d: %.3f'%(epoch, current_loss / n))
            
            print('Training process has finished. Saving trained model.')

            print('Start Testing')
            save_path = f'/content/drive/Shareddrives/NLP모델링/코드 연습/문서/model-fold-{fold}-c{c}.pth'
            torch.save(model.state_dict(), save_path)

            correct, total = 0, 0
            test = []
            pred = []
            with torch.no_grad():
                for i, data in enumerate(testloader, 0):
                    inputs, targets = data
                    outputs = model(inputs.to(device))
                    _, predicted = torch.max(outputs.data, 1)
                    _, target = torch.max(targets.data, 1)
                    total += targets.size(0)
                    correct += (predicted == target.to(device)).sum().item()
                    test.append(target)
                    pred.append(predicted)

                print('Accuracy for fold %d criteria%d: %d %%' % (fold, c, 100.0 * correct/total))
                print('-'*30)
                results[fold] = 100.0 * (correct / total)

                test = [j.item() for i in test for j in i.cpu()]
                pred = [j.item() for i in pred for j in i.cpu()]
                print('='*50)
                p, r, f = get_f1(test, pred, avg = 'macro')
                print('='*50)
                pre.append(p)
                rec.append(r)
                f1s.append(f)

            
        print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS for CRITERIA{c}')
        print('-'*30)
        s = 0.0
        for key, value in results.items():
            print(f'Fold {key} for Criteria{c}: {value} %')
            s += value
        print(f'Average : {s/len(results.items())} %')
        print(f'Average precision, recall, f1 : {sum(pre)/len(pre)}, {sum(rec) / len(rec)}, {sum(f1s) / len(f1s)}')

FOLD 1 for Criteria1
------------------------------
Reset trainable parameters of layer = Linear(in_features=100, out_features=20, bias=True)
Reset trainable parameters of layer = BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Linear(in_features=20, out_features=3, bias=True)
Loss after epoch     0: 1.059
Loss after epoch     1: 1.001
Loss after epoch     2: 0.965
Loss after epoch     3: 0.932
Loss after epoch     4: 0.905
Loss after epoch     5: 0.890
Loss after epoch     6: 0.878
Loss after epoch     7: 0.862
Loss after epoch     8: 0.845
Loss after epoch     9: 0.841
Loss after epoch    10: 0.829
Loss after epoch    11: 0.811
Loss after epoch    12: 0.822
Loss after epoch    13: 0.795
Loss after epoch    14: 0.798
Loss after epoch    15: 0.795
Loss after epoch    16: 0.808
Loss after epoch    17: 0.795
Loss after epoch    18: 0.791
Loss after epoch    19: 0.788
Loss after epoch    20: 0.776
Loss after epoch    21

KeyboardInterrupt: ignored

In [ ]:
criteria2 = pd.DataFrame(list(d2vnormdata['target']), columns = ['c1','c2','c3','c4','c5','c6'])
criteria2 = criteria2.astype('str')

c31 = LabelBinarizer().fit_transform(criteria2['c1'])
c32 = LabelBinarizer().fit_transform(criteria2['c2'])
c33 = LabelBinarizer().fit_transform(criteria2['c3'])
c34 = LabelBinarizer().fit_transform(criteria2['c4'])
c35 = LabelBinarizer().fit_transform(criteria2['c5'])
c36 = LabelBinarizer().fit_transform(criteria2['c6'])

criteria2['c1'] = list(c31)
criteria2['c2'] = list(c32)
criteria2['c3'] = list(c33)
criteria2['c4'] = list(c34)
criteria2['c5'] = list(c35)
criteria2['c6'] = list(c36)

In [ ]:
if __name__ == '__main__':

    k_folds = 5
    num_epochs = 50
    loss_function = nn.CrossEntropyLoss()

    results = {}

    # w2v_train1 = GetDataset(w2vaggdata);w2v_test1 = GetDataset(w2vaggdata)
    # w2v_train2 = GetDataset(w2vnormdata);w2v_test2 = GetDataset(w2vnormdata)
    # d2v_train1 = GetDataset(d2vnormdata);d2v_test1 = GetDataset(d2vnormdata)
    # w2v_dataset1 = ConcatDataset([w2v_train1, w2v_test1])
    # w2v_dataset2 = ConcatDataset([w2v_train2, w2v_test2])
    # d2v_dataset1 = ConcatDataset([d2v_train1, d2v_test1])
    for c in [1,2,3,4,5,6]:
        train = GetDataset(d2vnormdata, criteria2, 'c'+str(c));test = GetDataset(d2vnormdata, criteria2, 'c'+str(c))
        dataset = ConcatDataset([train, test])     

        kfold = KFold(n_splits = k_folds, shuffle = True)

        for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):
            print(f'FOLD {fold+1} for Criteria{c}')
            print('-' * 30)

            train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
            test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

            trainloader = DataLoader(dataset, batch_size = 10, sampler = train_subsampler)
            testloader = DataLoader(dataset, batch_size = 10, sampler = test_subsampler)

            model = nnn().to(device)
            model.apply(reset_weights)
        
            optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

            for epoch in range(0, num_epochs):

                current_loss = 0.0
                n = 0
                for i, data in enumerate(trainloader, 0):
                    inputs, targets = data
                    optimizer.zero_grad()
                    outputs = model(inputs.to(device))
                    loss = loss_function(outputs, targets.to(device))
                    loss.backward()
                    optimizer.step()

                    current_loss += loss.item()
                    n += 1
                    
                print('Loss after epoch %5d: %.3f'%(epoch, current_loss / n))
            
            print('Training process has finished. Saving trained model.')

            print('Start Testing')
            save_path = f'/content/drive/Shareddrives/NLP모델링/코드 연습/문서/model-fold-{fold}-c{c}.pth'
            torch.save(model.state_dict(), save_path)

            correct, total = 0, 0
            with torch.no_grad():
                for i, data in enumerate(testloader, 0):
                    inputs, targets = data
                    outputs = model(inputs.to(device))
                    _, predicted = torch.max(outputs.data, 1)
                    _, target = torch.max(targets.data, 1)
                    total += targets.size(0)
                    correct += (predicted == target.to(device)).sum().item()

                print('Accuracy for fold %d criteria%d: %d %%' % (fold, c, 100.0 * correct/total))
                print('-'*30)
                results[fold] = 100.0 * (correct / total)
            
        print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS for CRITERIA{c}')
        print('-'*30)
        sum = 0.0
        for key, value in results.items():
            print(f'Fold {key} for Criteria{c}: {value} %')
            sum += value
        print(f'Average : {sum/len(results.items())} %')

FOLD 1 for Criteria1
------------------------------
Reset trainable parameters of layer = Linear(in_features=100, out_features=64, bias=True)
Reset trainable parameters of layer = BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Linear(in_features=64, out_features=32, bias=True)
Reset trainable parameters of layer = BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Linear(in_features=32, out_features=16, bias=True)
Reset trainable parameters of layer = BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Linear(in_features=16, out_features=8, bias=True)
Reset trainable parameters of layer = BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
Reset trainable parameters of layer = Linear(in_features=8, out_features=3, bias=True)
Loss after epoch     0: 0.978
Loss